In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from scipy.optimize import brute
from decimal import Decimal, getcontext
from statsmodels.tsa.stattools import coint
import os

In [2]:

directory_path = '/Users/manasmaskar/Rutgers/Fall24/Algo Trading/HomeWork/HW1/DataFiles'

# List all files in the directory (assuming they are CSV files)
files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

# Initialize a dictionary to store each DataFrame, with filenames as keys
data_dict = {}

# Loop over each file and read it into a DataFrame
for file in files:
    file_path = os.path.join(directory_path, file)
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Use the filename (without extension) as the key in the dictionary
    data_dict[file] = df

# Check if all files have been loaded successfully
print(f"Loaded {len(data_dict)} files.")

# Example: Access one of the DataFrames (e.g., for the first file)
print(data_dict[files[0]].head())  # Display the first few rows of the first DataFrame


Loaded 29 files.
                         Local time     Open     High      Low    Close  \
0  01.01.2020 00:00:00.000 GMT-0500  1.29853  1.29853  1.29853  1.29853   
1  01.01.2020 01:00:00.000 GMT-0500  1.29853  1.29853  1.29853  1.29853   
2  01.01.2020 02:00:00.000 GMT-0500  1.29853  1.29853  1.29853  1.29853   
3  01.01.2020 03:00:00.000 GMT-0500  1.29853  1.29853  1.29853  1.29853   
4  01.01.2020 04:00:00.000 GMT-0500  1.29853  1.29853  1.29853  1.29853   

   Volume  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4     0.0  


In [3]:
#Clean the Local time by removing the GMT - 5:00
import re

# Loop through all dataframes in the dictionary and clean the 'Local time' column
for file, df in data_dict.items():
    # Use regex to remove any GMT offset (e.g., GMT-0500, GMT-0400, etc.)
    df['Time'] = df['Local time'].str.replace(r' GMT[+-]\d{4}', '', regex=True)
    
    # Convert the cleaned datetime string to a proper pandas datetime object
    df['Time'] = pd.to_datetime(df['Time'], format='%d.%m.%Y %H:%M:%S.%f')
    
    # Drop the old 'Local time' column if you no longer need it
    df.drop(columns=['Local time'], inplace=True)

# Check the cleaned data
print(data_dict[files[0]].head())  # Display the first few rows of the first DataFrame after cleaning



      Open     High      Low    Close  Volume                Time
0  1.29853  1.29853  1.29853  1.29853     0.0 2020-01-01 00:00:00
1  1.29853  1.29853  1.29853  1.29853     0.0 2020-01-01 01:00:00
2  1.29853  1.29853  1.29853  1.29853     0.0 2020-01-01 02:00:00
3  1.29853  1.29853  1.29853  1.29853     0.0 2020-01-01 03:00:00
4  1.29853  1.29853  1.29853  1.29853     0.0 2020-01-01 04:00:00


In [4]:
# Extracting the close prices in each df

for file, df in data_dict.items():
    # Extract the file name without the extension to use as the new DataFrame name
    df_name = file.split('.')[0] + '_df'
    
    # Select only the 'Cleaned Local Time' and 'Close' columns
    filtered_df = df[['Time', 'Close']].copy()
    
    # Store the filtered DataFrame in a variable with the dynamic name
    globals()[df_name] = filtered_df
    
    # Optionally, print the first few rows to confirm
    print(f"{df_name}:")
    print(globals()[df_name].head(), "\n")


USDCAD_Candlestick_1_Hour_BID_01_df:
                 Time    Close
0 2020-01-01 00:00:00  1.29853
1 2020-01-01 01:00:00  1.29853
2 2020-01-01 02:00:00  1.29853
3 2020-01-01 03:00:00  1.29853
4 2020-01-01 04:00:00  1.29853 

UKGILT_df:
                 Time    Close
0 2020-01-01 00:00:00  131.274
1 2020-01-01 01:00:00  131.274
2 2020-01-01 02:00:00  131.274
3 2020-01-01 03:00:00  131.274
4 2020-01-01 04:00:00  131.274 

AUDUSD_Candlestick_1_Hour_BID_01_df:
                 Time    Close
0 2020-01-01 00:00:00  0.70115
1 2020-01-01 01:00:00  0.70115
2 2020-01-01 02:00:00  0.70115
3 2020-01-01 03:00:00  0.70115
4 2020-01-01 04:00:00  0.70115 

BRENT_df:
                 Time   Close
0 2020-01-01 00:00:00  65.995
1 2020-01-01 01:00:00  65.995
2 2020-01-01 02:00:00  65.995
3 2020-01-01 03:00:00  65.995
4 2020-01-01 04:00:00  65.995 

DOLLAR_df:
                 Time   Close
0 2020-01-01 00:00:00  96.134
1 2020-01-01 01:00:00  96.134
2 2020-01-01 02:00:00  96.134
3 2020-01-01 03:00:00  96.13

In [5]:
close_prices_df = pd.DataFrame()

# Loop through each filtered DataFrame in globals() (which contains all individual DataFrames)
for file, df in data_dict.items():
    # Extract the file name without extension to use as the column label
    df_name = file.split('.')[0] + '_df'
    
    # Set the 'Time' column as the index and add the 'Close' column to the close_prices_df
    filtered_df = globals()[df_name]
    filtered_df.set_index('Time', inplace=True)
    
    # Add the 'Close' column with the file name as the column label
    close_prices_df[df_name] = filtered_df['Close']

# Calculate the pairwise correlation matrix for the close prices
correlation_matrix = close_prices_df.corr()

# Display the correlation matrix
print("Pairwise Correlation Matrix:")
print(correlation_matrix)

Pairwise Correlation Matrix:
                                     USDCAD_Candlestick_1_Hour_BID_01_df  \
USDCAD_Candlestick_1_Hour_BID_01_df                             1.000000   
UKGILT_df                                                      -0.449022   
AUDUSD_Candlestick_1_Hour_BID_01_df                            -0.891733   
BRENT_df                                                       -0.200071   
DOLLAR_df                                                       0.645871   
XAUUSD_Candlestick_1_Hour_BID_01_df                             0.280521   
DIESEL_df                                                      -0.032825   
COCOA_df                                                        0.403156   
BTCUSD_Candlestick_1_Hour_BID_01_df                            -0.265420   
NZDUSD_Candlestick_1_Hour_BID_01_df                            -0.860515   
SUGAR_df                                                        0.221588   
USA500_df                                                  

In [6]:
import plotly.express as px

# Calculate the pairwise correlation matrix for the close prices
correlation_matrix = close_prices_df.corr()

# Create a heatmap using Plotly
fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                color_continuous_scale="Viridis")

# Update layout for better readability
fig.update_layout(title="Pairwise Correlation Matrix of Close Prices",
                  xaxis_title="Instruments",
                  yaxis_title="Instruments",
                  width=800, height=800)

# Show the plot
fig.show()


<h1>Part 2 Cointegration

In [13]:
from statsmodels.tsa.stattools import coint
import pandas as pd

# Initialize an empty list to store cointegration results
cointegration_results = []

# List of instrument names  
instruments = list(data_dict.keys())

# Loop through each pair of instruments
for i in range(len(instruments)):
    for j in range(i + 1, len(instruments)):
        instrument1 = instruments[i]
        instrument2 = instruments[j]
        
        # Extract the 'Close' prices for both instruments
        close_prices_1 = data_dict[instrument1]['Close']
        close_prices_2 = data_dict[instrument2]['Close']
        
        # Perform the Engle-Granger Cointegration Test
        score, p_value, _ = coint(close_prices_1, close_prices_2)
        
        # Append the results: instrument1, instrument2, p-value
        cointegration_results.append((instrument1, instrument2, p_value))

# Sort the results by p-value (smallest p-value first)
cointegration_results_sorted = sorted(cointegration_results, key=lambda x: x[2])

# Display the top 10 most cointegrated pairs
print("Top 10 Most Cointegrated Pairs:")
for pair in cointegration_results_sorted[:10]:
    print(f"{pair[0]} vs {pair[1]}: p-value = {pair[2]:.4f}")


Top 10 Most Cointegrated Pairs:
BRENT.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv vs LIGHT.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv: p-value = 0.0001
XAUUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv vs COCOA.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv: p-value = 0.0110
ETHUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv vs COPPER.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv: p-value = 0.0158
BRENT.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv vs DIESEL.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv: p-value = 0.0213
AUDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv vs NZDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv: p-value = 0.0470
USSC2000.IDXUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv vs COPPER.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv: p-value = 0.0490
DIESEL.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv vs LIGHT.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.

In [ ]:
#code to save the selected df for their separate analysis
import os

# Example fetching for each DataFrame (adjust the names as needed)
BRENT_df = data_dict['BRENT.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv'][['Time', 'Close']]
LIGHT_df = data_dict['LIGHT.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv'][['Time', 'Close']]
XAUUSD_Candlestick_1_Hour_BID_01_df = data_dict['XAUUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv'][['Time', 'Close']]
COCOA_df = data_dict['COCOA.CMDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv'][['Time', 'Close']]

# Create a folder to store the CSV files (if it doesn't already exist)
folder_name = "cointegrated_pairs_data"
os.makedirs(folder_name, exist_ok=True)

# Save the DataFrames with the Time column
BRENT_df.to_csv(os.path.join(folder_name, 'BRENT_df.csv'), index=False)
LIGHT_df.to_csv(os.path.join(folder_name, 'LIGHT_df.csv'), index=False)
XAUUSD_Candlestick_1_Hour_BID_01_df.to_csv(os.path.join(folder_name, 'XAUUSD_Candlestick_1_Hour_BID_01_df.csv'), index=False)
COCOA_df.to_csv(os.path.join(folder_name, 'COCOA_df.csv'), index=False)

print("DataFrames with Time column saved successfully in the folder:", folder_name)



<h1> 3rd Part

In [ ]:
#Code to export AUDUSD... df for their analysis 3rd Part

AUDUSD_Candlestick_1_Hour_BID_01_df = data_dict['AUDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv'][['Time', 'Close']]
NZDUSD_Candlestick_1_Hour_BID_01_df = data_dict['NZDUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv'][['Time', 'Close']]


# Create a folder to store the CSV files (if it doesn't already exist)
folder_name = "cointegrated_pairs_data"
os.makedirs(folder_name, exist_ok=True)

# Save the DataFrames with the Time column
AUDUSD_Candlestick_1_Hour_BID_01_df.to_csv(os.path.join(folder_name, 'AUDUSD_df.csv'), index=False)
NZDUSD_Candlestick_1_Hour_BID_01_df.to_csv(os.path.join(folder_name, 'NZDUSD_df.csv'), index=False)